In [40]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import dill


In [41]:
#load trained model
model = load_model('model.h5')

#load encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('label_encoder_default.pkl','rb') as file:
    label_encoder_default = pickle.load(file)
    
with open('onehot_encoder_intent.pkl','rb') as file:
    one_encoder_intent = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)
    
with open('custom_encoding_house.pkl','rb') as file:
    custom_house = dill.load(file)
    
with open('custom_encoding_education.pkl','rb') as file:
    custom_education = dill.load(file)    

In [42]:
input = {'person_age':22.0,
         'person_gender':'female',
         'person_education':'Master',
         'person_income':71948.0,
         'person_emp_exp':0,
         'person_home_ownership':'RENT',
         'loan_amnt':35000.0,
         'loan_intent':'PERSONAL',
         'loan_int_rate':16.02,
         'loan_percent_income':0.49,
         'cb_person_cred_hist_length':3.0,
         'credit_score':561,
         'previous_loan_defaults_on_file':'No'}


In [43]:
input_df = pd.DataFrame([input])

In [44]:
input_df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No


In [45]:
encoded = one_encoder_intent.transform(input_df[['loan_intent']]).toarray()
encoded

array([[0., 0., 0., 0., 1., 0.]])

In [46]:
encoded_df = pd.DataFrame(encoded,columns=one_encoder_intent.get_feature_names_out(['loan_intent']))
encoded_df

,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,0.0,0.0,0.0,0.0,1.0,0.0


In [47]:
input_df = pd.concat([input_df.drop(['loan_intent'],axis=1), encoded_df],axis=1)
input_df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,22.0,female,Master,71948.0,0,RENT,35000.0,16.02,0.49,3.0,561,No,0.0,0.0,0.0,0.0,1.0,0.0


In [48]:
input_df['person_gender'] = label_encoder_gender.transform(input_df['person_gender'])
input_df['previous_loan_defaults_on_file'] = label_encoder_default.transform(input_df['previous_loan_defaults_on_file'])
input_df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,22.0,0,Master,71948.0,0,RENT,35000.0,16.02,0.49,3.0,561,0,0.0,0.0,0.0,0.0,1.0,0.0


In [49]:
input_df['person_education'] = custom_education(input_df)
input_df['person_home_ownership'] = custom_house(input_df)
input_df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,22.0,0,3,71948.0,0,0,35000.0,16.02,0.49,3.0,561,0,0.0,0.0,0.0,0.0,1.0,0.0


In [50]:
scaled = scaler.transform(input_df)
scaled

array([[-0.95473963, -1.10659687,  1.49509933, -0.10170961, -0.89233902,
        -0.87618587,  4.02990062,  1.67830446,  4.02454183, -0.73804283,
        -1.42527103, -1.01415574, -0.43512714, -0.50784406, -0.34418596,
        -0.48336029,  2.22938316, -0.45732836]])

In [51]:
pred = model.predict(scaled)
pred

1/1 [==============================] - 0s 53ms/step


array([[1.]], dtype=float32)

In [52]:
prob = pred[0][0]
prob

1.0

In [53]:
if prob > 0.5:
    print("Loan is  likely to be repaid")
else:
    print("Loan will not be repaid")


Loan is  likely to be repaid
